Model simulation Strain_4 (Knock-out sucA). By knockint-out sucA we delete another reaction that consume 2-oxoglutarate.

In [1]:
from cobra.io import read_sbml_model
model = read_sbml_model('modified_model.xml')
model

Name,iJO1366
Memory address,28d408e3a50
Number of metabolites,1810
Number of reactions,2592
Number of genes,1368
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


Based on the paper another knock-out that will increase 2-oxoglutarate concentration is sucA 


In [2]:
# research of sucA gene in our model
sucA = None
for gene in model.genes:
    if "sucA" in gene.name:
        sucA = gene.id
        break

print("Gene ID for sucA in model:", sucA)

Gene ID for sucA in model: b0726


In [3]:
model.genes.b0726 

Gene identifier,b0726
Name,sucA
Memory address,0x28d726b4f90
Functional,True
In 1 reaction(s),AKGDH


Let's search for the connected reaction to konck it out

In [4]:
model.reactions.AKGDH


Reaction identifier,AKGDH
Name,2-Oxogluterate dehydrogenase
Memory address,0x28d72cfcfd0
Stoichiometry,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c 2-Oxoglutarate + Coenzyme A + Nicotinamide adenine dinucleotide --> CO2 CO2 + Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA
GPR,b0116 and b0726 and b0727
Lower bound,0.0
Upper bound,1000.0


Test the production of EFE reaction before the knock-out

In [5]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9823718127269851 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.780573951434883 mmol/gDW*h


Knock-out of the reaction connected to sucA gene

In [6]:
model.reactions.AKGDH.knock_out()
model.reactions.AKGDH


Reaction identifier,AKGDH
Name,2-Oxogluterate dehydrogenase
Memory address,0x28d72cfcfd0
Stoichiometry,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c 2-Oxoglutarate + Coenzyme A + Nicotinamide adenine dinucleotide --> CO2 CO2 + Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA
GPR,b0116 and b0726 and b0727
Lower bound,0
Upper bound,0


Maximum theoretical biomass and EFE productivity after the knock-out

In [7]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9777223396312746 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.780573951434809 mmol/gDW*h


Maximum theoretical EFE productivity and yield after the knock-out

In [8]:
model.objective = model.reactions.eth_tp
eth_production = model.optimize().objective_value
print('Maximum productivity =', eth_production, 'mmol/gDW*h')
   
eth_maximum_yield = eth_production / (-1*(model.reactions.EX_glc__D_e.flux))
print('Maximum theoretical yield =', eth_maximum_yield, 'mmol-eth/mmol-glc')

Maximum productivity = 12.780573951434897 mmol/gDW*h
Maximum theoretical yield = 1.2780573951434897 mmol-eth/mmol-glc
